# Interactive Hodgkin-Huxley Interactive Simulation

The Hodgkin-Huxley (HH) model is a foundational mathematical model in computational neuroscience that describes how action potentials (spikes) are initiated and propagated in a neuron. It models the neuron's membrane as an electrical circuit, where the flow of ions (primarily Sodium ($\text{Na}^+$) and Potassium ($\text{K}^+$)) across the membrane is controlled by voltage-gated ion channels.

The core of the model is a system of ordinary differential equations (ODEs) that relate the membrane voltage ($V_m$) to the currents ($I$) passing through the channels:

$$C_m \frac{dV_m}{dt} = I_{app} - I_{Na} - I_K - I_L$$

Where $C_m$ is the membrane capacitance, $I_{app}$ is the injected current, and $I_{Na}$, $I_K$, and $I_L$ are the Sodium, Potassium, and Leak currents, respectively.


---


This notebook is designed to let you explore the dynamic mechanisms of the HH model by observing:
- Injected Current ($I_{app}$): How adjusting the **injected current** changes the neuron response.
- Ion Channel Dynamics: How Sodium ($\text{Na}^+$) and Potassium ($\text{K}^+$) currents shapes the neural spike.
- Gating Variables ($m$, $h$, $n$): How these gating variables control the open/closed states of the $\text{Na}^+$ and $\text{K}^+$ channels (activation/inactivation).


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
import ipywidgets as widgets

%matplotlib inline


### Model Parameters (Biological Constants)

These parameters are constants derived from the original 1952 squid axon data.
- `g_Na`, `g_K`, and `g_L` represent the **Maximum Conductances** (in $\text{mS}/\text{cm}^2$ or $\text{mmohs}/\text{cm}^2$). They control how easily each ion flows when the channels are fully open.
- `E_Na`, `E_K`, and `E_L` are the **Nernst Equilibrium Potentials** (in $\text{mV}$). These are the voltages each ion "wants" to reach, driving the current flow.

In [2]:
# Membrane capacitance
C_m  = 1.0   # uF/cm²

# Maximum conductances (how leaky each channel is when open)
g_Na = 120.0  # Sodium (Na⁺)
g_K  = 36.0   # Potassium (K⁺)
g_L  = 0.3    # Leak (mainly Cl⁻)

# Reversal potentials (in mV)
E_Na = 50.0    # Where Na⁺ equilibrium potential
E_K  = -77.0   # Where K⁺ equilibrium potential
E_L  = -54.387 # Leak equilibrium potential

## Gating Variable Functions

Each ion channel opens and closes based on voltage. These $\alpha$ (alpha, **opening rate**) and $\beta$ (beta, *+closing rate**) functions describe the **kinetic** rates of the gating variables.
- The **steady-state value** ($\text{X}_{\infty} = \alpha_x / (\alpha_x + \beta_x)$) is the expected final state.
- The **time constant** ($\tau_x = 1 / (\alpha_x + \beta_x)$) determines how fast the variable reaches that state.

These rates govern the dynamics of:
- `m`: Na⁺ activation gate
- `h`: Na⁺ inactivation gate
- `n`: K⁺ activation gate


In [3]:
def alpha_n(V): return 0.01*(V + 55)/(1 - np.exp(-(V + 55)/10))
def beta_n(V): return 0.125*np.exp(-(V + 65)/80)

def alpha_m(V): return 0.1*(V + 40)/(1 - np.exp(-(V + 40)/10))
def beta_m(V): return 4.0*np.exp(-(V + 65)/18)

def alpha_h(V): return 0.07*np.exp(-(V + 65)/20)
def beta_h(V): return 1/(1 + np.exp(-(V + 35)/10))

## Simulating the Neuron (Numerical Integration)

This function solves the system of ODEs (Ordinary Differential Equations) over time in response to a constant input current (`I_inj`).

We use the **Explicit Euler Method** for numerical integration. While simple, it requires a small timestep ($\text{dt}$) for stability and accuracy.

Each Timestep ($\text{dt}$):
- **Gating Variables** `m`, `h`, `n` are updated using their respective $\alpha$ and $\beta$ rates.
- **Ion Currents** `I_Na`, `I_K`, `I_L` are calculated based on the gating variables and the membrane voltage.
- **Membrane Voltage** `V` is updated based on the total net current (Ohm's Law: $\Delta V = (\text{Current} \times \text{Time}) / C_m$).

In [4]:
def simulate(I_inj=10.0, t_max=50, dt=0.01):
    time = np.arange(0, t_max, dt)
    n_steps = len(time)

    # Initialize state variables
    V = np.full(n_steps, -65.0)
    m = np.zeros(n_steps)
    h = np.zeros(n_steps)
    n = np.zeros(n_steps)

    # Initial gating values are set to their steady-state values (X_inf) at the initial resting voltage V[0] = -65 mV.
    m[0] = alpha_m(V[0]) / (alpha_m(V[0]) + beta_m(V[0]))
    h[0] = alpha_h(V[0]) / (alpha_h(V[0]) + beta_h(V[0]))
    n[0] = alpha_n(V[0]) / (alpha_n(V[0]) + beta_n(V[0]))

    # Currents for plotting
    I_Na = np.zeros(n_steps)
    I_K = np.zeros(n_steps)
    I_L = np.zeros(n_steps)

    for i in range(1, n_steps):
        # Update gating variables using Euler's approximation: dX/dt = alpha*(1-X) - beta*X
        # X[i] = X[i-1] + dt * (dX/dt)
        m[i] = m[i-1] + dt * (alpha_m(V[i-1]) * (1 - m[i-1]) - beta_m(V[i-1]) * m[i-1])
        h[i] = h[i-1] + dt * (alpha_h(V[i-1]) * (1 - h[i-1]) - beta_h(V[i-1]) * h[i-1])
        n[i] = n[i-1] + dt * (alpha_n(V[i-1]) * (1 - n[i-1]) - beta_n(V[i-1]) * n[i-1])

        # Compute currents using Ohm's Law (I_ion = g * X^p * (V - E_ion))
        I_Na[i] = g_Na * m[i]**3 * h[i] * (V[i-1] - E_Na)
        I_K[i]  = g_K  * n[i]**4 * (V[i-1] - E_K)
        I_L[i]  = g_L  * (V[i-1] - E_L)

        # Total net ionic current
        I_ion = I_inj - I_Na[i] - I_K[i] - I_L[i]

        # Update voltage using the primary HH equation (Euler's method): dV/dt = I_ion / C_m
        # V[i] = V[i-1] + dt * (dV/dt)
        V[i] = V[i-1] + dt * I_ion / C_m

    return time, V, m, h, n, I_Na, I_K, I_L


## Plotting

We visualize:
1. The changing **membrane voltage** over time
2. The **gating variables** that control ion channels
3. The individual **ion currents**

This helps us understand how a neuron's action potential is generated from microscopic channel dynamics.

In [7]:
def plot_hh(I_inj):
    time, V, m, h, n, I_Na, I_K, I_L = simulate(I_inj=I_inj)

    fig, axs = plt.subplots(3, 1, figsize=(15, 10), sharex=True)
    fig.suptitle(f'Hodgkin-Huxley Simulation (Injected Current: {I_inj:.2f} uA/cm²)', fontsize=16)

    axs[0].plot(time, V, label='Membrane Potential (mV)')
    axs[0].set_ylabel('V (mV)')
    axs[0].legend()

    axs[1].plot(time, m, label='m')
    axs[1].plot(time, h, label='h')
    axs[1].plot(time, n, label='n')
    axs[1].set_ylabel('Gating Variables')
    axs[1].legend()

    axs[2].plot(time, I_Na, label='I_Na')
    axs[2].plot(time, I_K, label='I_K')
    axs[2].plot(time, I_L, label='I_L')
    axs[2].set_ylabel('Currents (uA/cm²)')
    axs[2].set_xlabel('Time (ms)')
    axs[2].legend()

    plt.tight_layout()
    plt.show()


## Try It Yourself!

Use the slider to change the input current and observe how the neuron's behavior changes.

- Small current: no spike
- Medium current: single spike
- High current: multiple spikes (repetitive firing)

In [11]:
interact(plot_hh, I_inj=FloatSlider(min=-0, max=100, step=1, value=10, description='Injected Current'))

interactive(children=(FloatSlider(value=10.0, description='Injected Current', step=1.0), Output()), _dom_class…

<function __main__.plot_hh(I_inj)>

## Expected Simulation Results

This simulation produces three synchronized plots that illustrate the biophysical basis of the action potential. Use the **Injected Current ($\mathbf{I}_{inj}$)** slider to observe the key dynamics:

| $\mathbf{I}_{inj}$ Level | $\mathbf{V}$ (Membrane Potential) | $\mathbf{I}_{Na}$ (Sodium Current) | $\mathbf{I}_{K}$ (Potassium Current) |
| :--- | :--- | :--- | :--- |
| **Low (Sub-threshold)** | Slight depolarization, returns to rest. | No significant spike. | No significant activity. |
| **Medium (Threshold)** | Single, full action potential (spike). | Sharp, large **inward (negative)** spike. | **Delayed, outward (positive)** hump. |
| **High (Supra-threshold)** | Continuous train of repetitive firing. | Repeated sharp, **inward** spikes. | Repeated **delayed, outward** humps. |


### Plot Explanations (Dynamics to Watch)

1.  **Membrane Potential ($\mathbf{V}$):** This trace shows the neuron's all-or-none response.
    * Find the **threshold**: the minimum $\text{I}_{inj}$ required to trigger a full action potential ($\sim-65\text{mV}$ to $+40\text{mV}$).
    * If $\text{I}_{inj}$ is high, the neuron will fire repeatedly, with the firing rate depending on the recovery time of the ion channels.

2.  **Gating Variables ($\mathbf{m}, \mathbf{h}, \mathbf{n}$):** These curves (between 0 and 1) represent the fraction of channels that are open.
    * **$\mathbf{m}$ (Na activation)** moves the fastest, rising immediately to open the $\text{Na}^+$ channel.
    * **$\mathbf{h}$ (Na inactivation)** moves slowly and drops to close the $\text{Na}^+$ channel.
    * **$\mathbf{n}$ (K activation)** moves slowly and is delayed, rising after the spike begins to open the $\text{K}^+$ channel.

3.  **Ion Currents ($\mathbf{I}_{Na}, \mathbf{I}_{K}, \mathbf{I}_{L}$):** These show the flow of ions driving the voltage change.
    * **$\mathbf{I}_{Na}$** is a large, **inward (negative)** current that causes the rapid depolarization. It shuts off quickly.
    * **$\mathbf{I}_{K}$** is a **delayed, outward (positive)** current that pulls the voltage back down (repolarization).
    * The total current dictates the voltage change: $\Delta V \propto (\mathbf{I}_{inj} - \mathbf{I}_{Na} - \mathbf{I}_{K} - \mathbf{I}_{L})$.